In [ ]:
#imports
import pickle
import glob
import nibabel as nib
import numpy as np
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


import os
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt

print(tf.version.VERSION)

In [ ]:
#paths
USER = "leila"
CHECKPOINT_NAME = f"test_3_samp_5_epoch.ckpt"
CHECKPOINT_PATH = f"/autofs/space/celer_001/users/{USER}/ckpt/{CHECKPOINT_NAME}"
TRACER = "pbr28"
FINAL_DATA_PATH_FOR_MODEL = f"/autofs/space/celer_001/users/{USER}/data/{TRACER}"
FILTERED_PATIENT_LIST_WITH_GT = f"/autofs/space/celer_001/users/{USER}/working_{TRACER}/pickles/unfiltered_patient_list.pkl"


In [ ]:
def generate_file_list_object(filtered_patient_list_path, input_data_root, trial_time="3600-180", recon_alg="OP"):
    list_dataset_train = []

    with open(filtered_patient_list_path, "rb") as f:
        gt_patient_list = pickle.load(f)

    current_patients = [pat.split("/")[-2]  for pat in glob.glob(f"{input_data_root}/**/")]

    for patient in gt_patient_list:
        if patient in current_patients:
            list_dataset_train.append({
            'input' : [ f"{input_data_root}/{patient}/pet_nifti/{trial_time}_{recon_alg}.nii.gz", 
            f"{input_data_root}/{patient}/mr_nifti/t1_img_registered.nii.gz"
            ],
            'gt':f"{input_data_root}/{patient}/pet_nifti/gt_recon.nii.gz"
            })

    return list_dataset_train

def prepare_data_from_nifti(path_load, list_augments=[], scale_by_norm=True):
	# get nifti
	nib_load = nib.load(path_load)
	print(nib_load.header)
	# get data
	data_load = np.squeeze(nib_load.get_fdata())
	# transpose to slice*x*y*channel
	print("SHAPE", data_load.shape)
	data_load = np.transpose(data_load[:,:,:,np.newaxis], [2,0,1,3])
	# scale
	if scale_by_norm:
		#df = data_load.flatten()
		#norm_factor = np.linalg.norm(df)
		data_load = data_load / np.linalg.norm(data_load.flatten())
	# finish loading data
	print('loaded from {0}, data size {1} (sample, x, y, channel)'.format(path_load, data_load.shape))    
	
	# augmentation
	if len(list_augments)>0:
		print('data augmentation')
		list_data = []
		for augment in list_augments:
			print(augment)
			# data_augmented = augment_data(data_load, axis_xy = [1,2], augment = augment)
			# list_data.append(data_augmented.reshape(data_load.shape))
		# data_load = np.concatenate(list_data, axis = 0)
	return data_load #, norm_factor # KC 20171018


	

In [ ]:
import numpy as np
# use skimage metrics
from skimage.metrics import mean_squared_error, normalized_root_mse, peak_signal_noise_ratio, structural_similarity

# psnr with TF
try:
	from tensorflow.python.keras import backend as K
	from tensorflow.compat.v1 import log as tf_log
	from tensorflow.compat.v1 import constant as tf_constant
	import tensorflow.compat.v1 as tf
except:
	print('import keras and tf backend failed')

def PSNRLoss(y_true, y_pred):
	"""
	PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.

	It can be calculated as
	PSNR = 20 * log10(MAXp) - 10 * log10(MSE)

	When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
	However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
	Thus we remove that component completely and only compute the remaining MSE component.
	"""
	try:
		#use theano
		return 20.*np.log10(K.max(y_true)) -10. * np.log10(K.mean(K.square(y_pred - y_true)))
	except:
		denominator = tf_log(tf_constant(10.0))
		return 20.*tf_log(K.max(y_true)) / denominator -10. * tf_log(K.mean(K.square(y_pred - y_true))) / denominator
	return 0

#get error metrics, for psnr, ssimr, rmse, score_ismrm
def getErrorMetrics(im_pred, im_gt, mask = None):
	# flatten array
	im_pred = np.array(im_pred).astype(np.float).flatten()
	im_gt = np.array(im_gt).astype(np.float).flatten()
	if mask is not None:
		mask = np.array(mask).astype(np.float).flatten()
		im_pred = im_pred[mask>0]
		im_gt = im_gt[mask>0]
	mask=np.abs(im_gt.flatten())>0

	# check dimension
	assert(im_pred.flatten().shape==im_gt.flatten().shape)

	# NRMSE
	try:
		rmse_pred = normalized_root_mse(im_gt, im_pred)
	except:
		rmse_pred = float('nan')

	# PSNR
	try:
		psnr_pred = peak_signal_noise_ratio(im_gt, im_pred)
	except:
		psnr_pred = float('nan')
		#psnr_pred = psnr(im_gt, im_pred)
		#print('use psnr')
	
	# ssim
	try:
		ssim_pred = structural_similarity(im_gt, im_pred)
		score_ismrm = sum((np.abs(im_gt.flatten()-im_pred.flatten())<0.1)*mask)/(sum(mask)+0.0)*10000
	except:
		ssim_pred = float('nan')
		score_ismrm = float('nan')
        
	return {'rmse':rmse_pred,'psnr':psnr_pred,'ssim':ssim_pred,'score_ismrm':score_ismrm}
	


In [ ]:

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Concatenate, Conv2D, Conv2DTranspose, BatchNormalization, Convolution2D, MaxPooling2D, UpSampling2D, Dense, concatenate
from keras.layers import Add as keras_add
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
from keras.losses import mean_absolute_error, mean_squared_error
from keras import backend as K
import numpy as np

# clean up
def clearKerasMemory():
    K.clear_session()

# use part of memory
def setKerasMemory(limit=0.3):
    from tensorflow.compat.v1 import ConfigProto as tf_ConfigProto
    from tensorflow.compat.v1 import Session as tf_Session
    from tensorflow.python.keras.backend import set_session
    config = tf_ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = limit
    set_session(tf_Session(config=config))

# encoder-deocder
def deepEncoderDecoder(num_channel_input=1, num_channel_output=1, 
	img_rows=128, img_cols=128, y=np.array([-1,1]), 
	lr_init=None, loss_function=mean_absolute_error, metrics_monitor=[PSNRLoss, mean_absolute_error, mean_squared_error],
	num_poolings = 3, num_conv_per_pooling = 3, 
	with_bn=False, verbose=1):
	# BatchNorm
	if with_bn:
		lambda_bn = lambda x: BatchNormalization()(x)
	else:
		lambda_bn = lambda x: x
	# layers
#     For 2D data (e.g. image), "tf" assumes (rows, cols, channels) while "th" assumes (channels, rows, cols).
	inputs = Input((img_rows, img_cols, num_channel_input))  
	if verbose:
		print(inputs)
	
	#step1
	conv1 = inputs
	num_channel_first = 32
	for i in range(num_conv_per_pooling):
		conv1 = Conv2D(num_channel_first, (3, 3), padding="same", activation="relu")(conv1)
		conv1 = lambda_bn(conv1)    
	pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
	if verbose:
		print(conv1,pool1)

	# encoder pools
	convs = [inputs, conv1]
	pools = [inputs, pool1]
	list_num_features = [num_channel_input, num_channel_first]
	for i in range(1, num_poolings):
		#step2	
		conv_encoder = pools[-1]
		num_channel = num_channel_first*(2**(i-1))
		for j in range(num_conv_per_pooling):
			conv_encoder = Conv2D(num_channel, (3, 3), padding="same", activation="relu")(conv_encoder)
			conv_encoder = lambda_bn(conv_encoder)    
		pool_encoder = MaxPooling2D(pool_size=(2, 2))(conv_encoder)
		if verbose:
			print(conv_encoder,pool_encoder)
		pools.append(pool_encoder)
		convs.append(conv_encoder)
		list_num_features.append(num_channel)

	# center connection
	conv_center = Conv2D(list_num_features[-1], (3, 3), padding="same", activation="relu",
					   kernel_initializer='zeros',
					   bias_initializer='zeros')(pools[-1])     
	conv_center = keras_add()([pools[-1], conv_center])				   
	conv_decoders = [conv_center]
	if verbose:
		print(conv_center)

	# decoder steps
	for i in range(1, num_poolings+1):
# 		print('decoder', i, convs, pools)
# 		print(UpSampling2D(size=(2, 2))(conv_center))
# 		print(convs[-i])
		up_decoder = concatenate([UpSampling2D(size=(2, 2))(conv_decoders[-1]), convs[-i]])	
		conv_decoder = up_decoder
		for j in range(num_conv_per_pooling):
			conv_decoder = Conv2D(list_num_features[-i], (3, 3), padding="same", activation="relu")(conv_decoder)
			conv_decoder = lambda_bn(conv_decoder)     
		conv_decoders.append(conv_decoder)
		if verbose:
			print(conv_decoder,up_decoder)        

	# output layer
	conv_decoder = conv_decoders[-1]
	if max(abs(y))<=1:
		if min(np.array(y).flatten())<0:
#         tanh -1~+1
			conv_output = Conv2D(num_channel_output, (1, 1), padding="same", activation="tanh")(conv_decoder)
			print('use tanh activation')
		else:
			conv_output = Conv2D(num_channel_output, (1, 1), padding="same", activation='sigmoid')(conv_decoder)    
			print('use sigmoid activation')
	else:
		conv_output = Conv2D(num_channel_output, (1, 1), padding="same", activation='linear')(conv_decoder)    
		print('use linear activation')
	if verbose:
		print(conv_output)
	
	# model
	model = Model(outputs=conv_output, inputs=inputs)
	if verbose:
		print(model)
	
	# fit
	if lr_init is not None:
		optimizer = Adam(lr=lr_init)#,0.001 rho=0.9, epsilon=1e-08, decay=0.0)
	else:
		optimizer = Adam()
	model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics_monitor)
	
	return model


In [ ]:
'''
augmentation
'''
list_augments = []
num_augment_flipxy = 2
num_augment_flipx = 2
num_augment_flipy = 2
num_augment_shiftx = 1
num_augment_shifty = 1
for flipxy in range(num_augment_flipxy):
	for flipx in range(num_augment_flipx):
		for flipy in range(num_augment_flipy):
			for shiftx in range(num_augment_shiftx):
				for shifty in range(num_augment_shifty):
					augment={'flipxy':flipxy,'flipx':flipx,'flipy':flipy,'shiftx':shiftx,'shifty':shifty}
					list_augments.append(augment)
list_augments = []
num_augment=len(list_augments)
print('will augment data with {0} augmentations'.format(num_augment))


will augment data with 0 augmentations


In [ ]:

list_dataset_train = generate_file_list_object(FILTERED_PATIENT_LIST_WITH_GT , FINAL_DATA_PATH_FOR_MODEL)
num_dataset_train = len(list_dataset_train)    
list_data_train_input = []


for index_data in range(num_dataset_train):
	# directory
	# headmask = prepare_data_from_nifti(os.path.dirname(list_dataset_train[index_data]['input'][1])+'/headmask_inv.nii', list_augments, False)
	headmask = prepare_data_from_nifti(os.path.dirname(list_dataset_train[index_data]['input'][1])+'/t1_mask_registered.nii.gz', list_augments, False)

	list_data_train_input = []
	for path_train_input in list_dataset_train[index_data]['input']:
		# load data
		data_train_input = prepare_data_from_nifti(path_train_input, list_augments)
		data_train_input = np.multiply(data_train_input, headmask) # data_train_input 
		list_data_train_input.append(data_train_input)
	data_train_input = np.concatenate(list_data_train_input, axis=-1)
	
	
	# load data ground truth
	path_train_gt = list_dataset_train[index_data]['gt']
	data_train_gt = prepare_data_from_nifti(path_train_gt, list_augments)

'''
setup parameters
'''


# related to model
num_poolings = 3
num_conv_per_pooling = 3
# related to training
lr_init = 0.0002
num_epoch = 100
ratio_validation = 0.1
validation_split = 0.1
batch_size = 4
y_range = [-0.5,0.5]
# default settings
num_channel_input = data_train_input.shape[-1]
num_channel_output = data_train_gt.shape[-1]
img_rows = data_train_input.shape[1]
img_cols = data_train_gt.shape[1]
keras_memory = 0.4
keras_backend = 'tf'
with_batch_norm = True
print('setup parameters')


'''
init model
'''
callback_checkpoint = ModelCheckpoint(CHECKPOINT_PATH, 
								monitor='val_loss', 
								save_best_only=True)
setKerasMemory(keras_memory)
model = deepEncoderDecoder(num_channel_input = num_channel_input,
						num_channel_output = num_channel_output,
						img_rows = img_rows,
						img_cols = img_cols,
						lr_init = lr_init, 
						num_poolings = num_poolings, 
						num_conv_per_pooling = num_conv_per_pooling, 
						with_bn = with_batch_norm, verbose=1)
print('train model:', CHECKPOINT_PATH)
print('parameter count:', model.count_params())

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 344 344 127   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.        2.08626   2.08626   2.031246  0.        0.        0.
  0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'Warped image using NiftyReg (reg_resample)'
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : -0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 357.973
qo

2022-06-15 15:05:26.209606: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-15 15:05:26.543071: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 15:05:26.543176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-15 15:05:26.543220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot ope

KerasTensor(type_spec=TensorSpec(shape=(None, 344, 344, 2), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 344, 344, 32), dtype=tf.float32, name=None), name='batch_normalization_2/FusedBatchNormV3:0', description="created by layer 'batch_normalization_2'") KerasTensor(type_spec=TensorSpec(shape=(None, 172, 172, 32), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")
KerasTensor(type_spec=TensorSpec(shape=(None, 172, 172, 32), dtype=tf.float32, name=None), name='batch_normalization_5/FusedBatchNormV3:0', description="created by layer 'batch_normalization_5'") KerasTensor(type_spec=TensorSpec(shape=(None, 86, 86, 32), dtype=tf.float32, name=None), name='max_pooling2d_1/MaxPool:0', description="created by layer 'max_pooling2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 86, 86, 64), dtype=tf.float32, name=None), name='batch_normalizat

/autofs/space/celer_001/users/leila/miniconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# with open('/autofs/space/celer_001/users/leila/ckpt/test_3_samp_5_epoch.ckpt/saved_model.pb') as f:
#     model = tf.keras.models.load_model(f)

model.load_weights("/autofs/space/celer_001/users/leila/ckpt/weights-001-0.0319.hdf5")


In [ ]:
class DataGenerator(object):
	'Generates data for Keras'
	def __init__(self, dim_x = 512, dim_y = 512, dim_z = 6, dim_output = 1, 
				batch_size = 2, shuffle = True, verbose = 1,
				scale_data = 1.0, scale_baseline = 1.0):
		'Initialization'
		self.dim_x = dim_x
		self.dim_y = dim_y
		self.dim_z = dim_z
		self.dim_output = dim_output
		self.batch_size = batch_size
		self.shuffle = shuffle
		self.verbose = verbose
		self.scale_data = scale_data
		self.scale_baseline = scale_baseline

	def generate(self, dir_sample, list_IDs):
		'Generates batches of samples'
		# Infinite loop
		while 1:
			# Generate order of exploration of dataset
			indexes = self.__get_exploration_order(list_IDs)
			if self.verbose>0:
				print('indexes:', indexes)
			# Generate batches
			imax = int(len(indexes)/self.batch_size)
			if self.verbose>0:            
				print('imax:', imax)
			for i in range(imax):
				# Find list of IDs
				list_IDs_temp = [list_IDs[k] for k in indexes[i*self.batch_size:(i+1)*self.batch_size]]
				if self.verbose>0:
					print('list_IDs_temp:', list_IDs_temp)
				# Generate data
				X, Y = self.__data_generation(dir_sample, list_IDs_temp)
				if self.verbose>0:                
					print('generated dataset size:', X.shape, Y.shape)

				yield X, Y

	def __get_exploration_order(self, list_IDs):
		'Generates order of exploration'
		# Find exploration order
		indexes = np.arange(len(list_IDs))
		if self.shuffle == True:
			np.random.shuffle(indexes)
		return indexes

	def __data_generation(self, dir_sample, list_IDs_temp, ext_data = 'npz'):
		'Generates data of batch_size samples' # X : (n_samples, v_size, v_size, v_size, n_channels)
		# Initialization
		X = np.empty((self.batch_size, self.dim_x, self.dim_y, self.dim_z, 1))
		Y = np.empty((self.batch_size, self.dim_x, self.dim_y, self.dim_output, 1))

		# Generate data
		for i, ID in enumerate(list_IDs_temp):
			# Store volume
			data_load = np.load(os.path.join(dir_sample, '{0}.{1}'.format(ID,ext_data)))
			X[i, :, :, :, 0] = data_load['input']
			Y[i, :, :, :, 0] = data_load['output'] 
		X = X[:,:,:,:,0]
		Y = Y[:,:,:,:,0]        
		X = X * self.scale_data
		Y = Y * self.scale_data
		Y = Y - self.scale_baseline * X[:,:,:,0:1]   
		print(X.shape, Y.shape)   
		return X, Y

In [ ]:
# details inside generator
params_generator = {'dim_x': img_rows,
		  'dim_y': img_cols,
		  'dim_z': num_channel_input,
		  'dim_output': num_channel_output,
		  'batch_size': 4,
		  'shuffle': False,
		  'verbose': 0,
		  'scale_data': 100.,
		  'scale_baseline': 1.0}
print('generator parameters:', params_generator)

generator parameters: {'dim_x': 344, 'dim_y': 344, 'dim_z': 2, 'dim_output': 1, 'batch_size': 4, 'shuffle': False, 'verbose': 0, 'scale_data': 100.0, 'scale_baseline': 1.0}


In [ ]:
''' 
setup train and val generator
'''
!pwd
dir_samples = '../../data/data_sample/'
validation_split = 0.1
ext_data = 'npz'
index_sample_total = len([x for x in os.listdir(dir_samples) if x.endswith(ext_data)])
list_indexes_train = np.random.permutation(index_sample_total)
if validation_split>1:
	list_indexes_val = list_indexes_train[-validation_split:].tolist()
	list_indexes_train = list_indexes_train[:int(index_sample_total-validation_split)].tolist()    
else:
	list_indexes_val = list_indexes_train[-int(index_sample_total*validation_split):].tolist()
	list_indexes_train = list_indexes_train[:int(index_sample_total*(1-validation_split))].tolist()
print('train on {0} samples and validation on {1} samples'.format(
		len(list_indexes_train), len(list_indexes_val)))
training_generator = DataGenerator(**params_generator).generate(dir_samples, list_indexes_train)
validation_generator = DataGenerator(**params_generator).generate(dir_samples, list_indexes_val)

/autofs/space/celer_001/users/leila/pbr28-low-high-count/model-results
train on 2743 samples and validation on 304 samples


In [ ]:
prediction = model.predict_generator(validation_generator, verbose=1, steps=100)

(4, 344, 344, 2) (4, 344, 344, 1)
(4, 344, 344, 2) (4, 344, 344, 1)
  1/100 [..............................] - ETA: 12s

/home/scratch/ipykernel_3639511/1026782263.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model.predict_generator(validation_generator, verbose=1, steps=100)


(4, 344, 344, 2) (4, 344, 344, 1)
100/100 [==============================] - 9s 95ms/step


In [ ]:
training_generator

NameError: name 'training_generator' is not defined